In [1]:
import pymongo
import time
import pandas as pd
import json
import nltk

# Conexão com o MongoDB

In [2]:
client = pymongo.MongoClient()

# database
db = client['IWD2021']  

# collection
collection = db.tweets_en

In [3]:
pipeline = [
    {
        '$project': {
            'tweet_id': '$id_str', 
            'date': {
                '$dateFromString': {
                    'dateString': '$created_at'
                }
            }, 
            'tweet_text': {
                '$ifNull': [
                    '$retweeted_status.full_text', '$full_text'
                ]
            }, 
            'num_likes': '$favorite_count', 
            'num_retweets': '$retweet_count', 
            'quoted_status_id': {
                '$ifNull': [
                    '$quoted_status_id_str', None
                ]
            }, 
            'retweeted_status_id': {
                '$ifNull': [
                    '$retweeted_status.id_str', None
                ]
            }, 
            'reply_to_user': {
                '$ifNull': [
                    '$in_reply_to_user_id_str', None
                ]
            }, 
            'reply_to_status': {
                '$ifNull': [
                    '$in_reply_to_status_id_str', None
                ]
            }, 
            'user_id': '$user.id_str', 
            'screen_name': '$user.screen_name', 
            'followers': '$user.followers_count', 
            'following': '$user.friends_count'
        }
    }, {
        '$project': {
            '_id': 0, 
            'tweet_id': 1, 
            'date': {
                '$dateToString': {
                    'format': '%d/%m/%Y %H:%M:%S', 
                    'date': '$date'
                }
            }, 
            'tweet_text': 1, 
            'num_likes': 1, 
            'num_retweets': 1, 
            'quoted_status_id': 1, 
            'retweeted_status_id': 1, 
            'reply_to_user': 1, 
            'reply_to_status': 1, 
            'user_id': 1, 
            'screen_name': 1, 
            'followers': 1, 
            'following': 1
        }
    }
]

In [4]:
time_start = time.time()
document = list(collection.aggregate(pipeline=pipeline))
time_end = time.time()

print(
    f'Tempo de execução: {time.strftime("%H:%M:%S", time.gmtime(time_end - time_start))}')
print(f'{len(document)} documentos selecionados')

Tempo de execução: 00:01:04
2459599 documentos selecionados


In [5]:
print(f'Amostra de um documento:\n')
print(json.dumps(document[0], sort_keys=False, indent=3, ensure_ascii=False))

Amostra de um documento:

{
   "tweet_id": "1368641229731360773",
   "tweet_text": "@MargaretEWard The irony that I can't attend any of these online events...because my kids aren't in school and I have to still home school them, is NOT lost on me 😂😭😂😭 Cheers for being in our corner, Margaret, while forging your own path too! #IWD2021",
   "num_likes": 4,
   "num_retweets": 0,
   "quoted_status_id": null,
   "retweeted_status_id": null,
   "reply_to_user": "17868868",
   "reply_to_status": "1368629846872580097",
   "user_id": "1240113597562458114",
   "screen_name": "HildaMcevoy",
   "followers": 445,
   "following": 1537,
   "date": "07/03/2021 19:14:31"
}


In [6]:
df = pd.DataFrame(document)
df = df[['tweet_id', 'date', 'tweet_text', 'num_likes', 'num_retweets',
                   'retweeted_status_id', 'quoted_status_id', 'reply_to_user', 'reply_to_status',
                   'user_id', 'screen_name', 'followers', 'following']]
df.head()

,tweet_id,date,tweet_text,num_likes,num_retweets,retweeted_status_id,quoted_status_id,reply_to_user,reply_to_status,user_id,screen_name,followers,following
0,1368641229731360773,07/03/2021 19:14:31,@MargaretEWard The irony that I can't attend a...,4,0,None,None,17868868,1368629846872580097,1240113597562458114,HildaMcevoy,445,1537
1,1368641259347320842,07/03/2021 19:14:39,There is no denying women have been disproport...,0,58,1367605354570342402,None,None,None,1255459296965529600,satyar2021,201,338
2,1368641305396408320,07/03/2021 19:14:49,"‘I am going to be a bit ranty today, because t...",3,0,None,None,None,None,3021736555,NoWineImFine,2318,983
3,1368641259284221954,07/03/2021 19:14:38,Who are the women that inspire you?\n\nOn the ...,0,700,1368601342307168256,None,None,None,3097354848,AlydiaSecosana,703,159
4,1368641318398885892,07/03/2021 19:14:53,THANK U GIRLZ #InternationalWomensDay https://...,0,78,1368625731165712385,None,None,None,1286755980794114048,glorydays___CB,102,131


In [7]:
print(f'Amostra de um tweet antes do pré-processamento:\n\n{df.tweet_text[0]}')

Amostra de um tweet antes do pré-processamento:

@MargaretEWard The irony that I can't attend any of these online events...because my kids aren't in school and I have to still home school them, is NOT lost on me 😂😭😂😭 Cheers for being in our corner, Margaret, while forging your own path too! #IWD2021


In [8]:
df.to_csv('tweets_en.csv')

### Pré-processamento

In [9]:
# remover links
df['clean_text'] = df['tweet_text'].str.replace(r"http\S+","") 

df.clean_text[0]

"@MargaretEWard The irony that I can't attend any of these online events...because my kids aren't in school and I have to still home school them, is NOT lost on me 😂😭😂😭 Cheers for being in our corner, Margaret, while forging your own path too! #IWD2021"

In [10]:
# remover mentions
df['clean_text'] = df['clean_text'].str.replace(r"@\S+","")

df.clean_text[0]

" The irony that I can't attend any of these online events...because my kids aren't in school and I have to still home school them, is NOT lost on me 😂😭😂😭 Cheers for being in our corner, Margaret, while forging your own path too! #IWD2021"

In [11]:
# remover hashtags
df['clean_text'] = df['clean_text'].str.replace(r"#\S+"," ")

df.clean_text[0]

" The irony that I can't attend any of these online events...because my kids aren't in school and I have to still home school them, is NOT lost on me 😂😭😂😭 Cheers for being in our corner, Margaret, while forging your own path too!  "

In [12]:
# remover quebra de linhas
df['clean_text'] = df['clean_text'].str.replace(r"\n"," ")

df.clean_text[0]

" The irony that I can't attend any of these online events...because my kids aren't in school and I have to still home school them, is NOT lost on me 😂😭😂😭 Cheers for being in our corner, Margaret, while forging your own path too!  "

In [13]:
# remover pontuações
df['clean_text'] = df['clean_text'].str.replace(r"[^\w\s#]|_"," ")

df.clean_text[0]

' The irony that I can t attend any of these online events   because my kids aren t in school and I have to still home school them  is NOT lost on me      Cheers for being in our corner  Margaret  while forging your own path too   '

In [14]:
# remover números
df['clean_text'] = df['clean_text'].str.strip().str.replace(r"\b(?:[0-9]*)\b","")

df.clean_text[0]

'The irony that I can t attend any of these online events   because my kids aren t in school and I have to still home school them  is NOT lost on me      Cheers for being in our corner  Margaret  while forging your own path too'

In [15]:
# remover non ascii caracteres
df['clean_text'] = df['clean_text'].str.strip().str.replace(r"[^a-zA-z0-9#|\s]","")

df.clean_text[0]

'The irony that I can t attend any of these online events   because my kids aren t in school and I have to still home school them  is NOT lost on me      Cheers for being in our corner  Margaret  while forging your own path too'

In [16]:
# remover espaços duplos
df['clean_text'] = df['clean_text'].str.strip().str.replace(r"\s{2,}"," ")

df.clean_text[0]

'The irony that I can t attend any of these online events because my kids aren t in school and I have to still home school them is NOT lost on me Cheers for being in our corner Margaret while forging your own path too'

### converter todas as letras para minúsculas

In [17]:
df['clean_text'] = df['clean_text'].str.lower()

df.clean_text[0]

'the irony that i can t attend any of these online events because my kids aren t in school and i have to still home school them is not lost on me cheers for being in our corner margaret while forging your own path too'

### remover letras que se repetem em sequência mais de 3x

In [18]:
import string
alphabet = list(string.ascii_lowercase)

for letter in alphabet:
    pattern = '{}{}'.format(letter, '{3,}')
    df['clean_text'] = df['clean_text'].str.replace(r'{}'.format(pattern), letter)
    
df.clean_text[0]

'the irony that i can t attend any of these online events because my kids aren t in school and i have to still home school them is not lost on me cheers for being in our corner margaret while forging your own path too'

### remover stpwords

In [19]:
with open('../stopwords/english.txt', 'r') as file:
    stopwords = file.readlines()
stopwords = [sw.replace('\n','') for sw in stopwords]

def remover_stopwords(texto):
    palavras = [i for i in texto.split() if not i in stopwords]
    return (" ".join(palavras))

df['clean_text'] = df['clean_text'].apply(remover_stopwords)

df.clean_text[0]

'irony attend online events kids school still home school lost cheers corner margaret forging path'

### remover palavras com 2 caracteres ou menos

In [20]:
def remover_len_2(texto):
    palavras = [i for i in texto.split() if len(i) > 2]
    return (" ".join(palavras))

df['clean_text'] = df['clean_text'].apply(remover_len_2)

df.clean_text[0]

'irony attend online events kids school still home school lost cheers corner margaret forging path'

In [21]:
print(f'Amostra de um tweet depois do pré-processamento:\n\n{df.clean_text[0][0]}')

Amostra de um tweet depois do pré-processamento:

i


In [22]:
def reduzir_radical(texto):
    stemmer = nltk.stem.RSLPStemmer()
    palavras = [stemmer.stem(i) for i in texto.split()]
    return (" ".join(palavras))

In [23]:
reduzir_radical(df.tweet_text[0])

"@margareteward the irony that i can't attend any of thes onlin events...becaus my kid aren't in school and i hav to still hom school them, is not lost on me 😂😭😂😭 che for being in our corner, margaret, whil forging your own path too! #iwd2021"

In [24]:
# df['tweet_text'] = df['tweet_text'].apply(reduzir_radical)

In [25]:
import unicodedata

def remover_acentuacao(text):
    # Unicode normalize transforma um caracter em seu equivalente em latin.
    nfkd = unicodedata.normalize('NFKD', text)
    palavra_sem_acento = u"".join([c for c in nfkd if not unicodedata.combining(c)])
    return (palavra_sem_acento)

In [26]:
remover_acentuacao(df.tweet_text[0])

"@MargaretEWard The irony that I can't attend any of these online events...because my kids aren't in school and I have to still home school them, is NOT lost on me 😂😭😂😭 Cheers for being in our corner, Margaret, while forging your own path too! #IWD2021"

In [27]:
# df['tweet_text'] = df['tweet_text'].apply(remover_acentuacao)

In [28]:
print(f'Amostra de um tweet depois do pré-processamento:\n\n{df.clean_text[0]}')

Amostra de um tweet depois do pré-processamento:

irony attend online events kids school still home school lost cheers corner margaret forging path


### Etapa opcional

O atributo 'lang' presentes nos documentos podem ter sido atribuidos erroneamente. Objetivando eliminar tweets que possam ser de um idioma não requisitado, desenvolveram-se 2 funções para ajudar a confirmar o idioma destes.

Para desabilitar warnings:

In [29]:
# import logging
# logging.disable(logging.WARNING)

#### 1. Usando polyglot

In [30]:
# import polyglot
# from polyglot.detect import Detector

In [31]:
# def get_polyglot_language(text):
#     try:
#         detector = Detector(text)
#     except polyglot.detect.base.UnknownLanguage:
#         lang = 'Undefined'
#     else:
#         lang = detector.language.name
    
#     return (lang)

In [32]:
# get_polyglot_language(df['tweet_text'][0])

In [33]:
# df['polyglot_lang'] = df['tweet_text'].apply(get_polyglot_language)

In [34]:
# df['polyglot_lang'].value_counts()

#### 2. Usando textblob

In [35]:
# import textblob
# from textblob import TextBlob
# import pycountry

In [36]:
# def get_textblob_language(text):
#     try:
#         time.sleep(0.2)
#         b = TextBlob(text)
#         language = pycountry.languages.get(alpha_2=b.detect_language())
#         language_name = language.name
#     except textblob.exceptions.TranslatorError:
#         # language_name = "Deu ruim"
#         language_name = 'Undefined'
    
#     return(language_name) 

In [37]:
# get_textblob_language(df['tweet_text'][0])

In [38]:
# df['textblob_lang'] = df['tweet_text'].apply(get_textblob_language)

Para reabilitar warnings:

In [39]:
# logging.disable(logging.NOTSET)

In [40]:
# df.shape

# Criação do arquivo .csv

### Cenário 1: base completa

Caso tenha se utilizado umas das duas funções - **get_polyglot_language**, por exemplo - use o seguinte trecho para gerar o arquivo .csv:

In [41]:
# index = df['polyglot_lang'] == 'Portuguese'
# new_df = df[index]
# new_df = new_df.drop('polyglot_lang', axis=1)

In [42]:
# new_df.to_csv('cenario1_8M2020_polyglot_tweets_pt.csv', index=False)

Caso contrário:

In [43]:
# df = df.drop('polyglot_lang', axis=1)

In [44]:
df = df[df['clean_text'] != '']

In [45]:
df.shape

(2452640, 14)

In [46]:
df.to_csv('cenario1_8M2021_tweets_en.csv', index=False)

criação do arquivo .txt (somente com os textos dos tweets):

In [47]:
import numpy as np

In [48]:
tweets_text = np.array(df['clean_text'])

In [49]:
np.savetxt(fname='cenario1_8M2021_tweets_en.txt', X=tweets_text, fmt='%s')

## Cenário 2: tweets sem duplicatas

In [50]:
df.drop_duplicates(['clean_text'], inplace = True)

In [51]:
df.shape

(557957, 14)

In [52]:
df.to_csv('cenario2_8M2021_tweets_en.csv', index=False)

In [53]:
tweets_text = np.array(df['clean_text'])

In [54]:
np.savetxt(fname='cenario2_8M2021_tweets_en.txt', X=tweets_text, fmt='%s')